In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# standard imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plte
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Je télécharge ma donnée et la regarde en détail

In [ ]:
labels_csv = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
print(labels_csv.describe())
print(labels_csv.head())

Je peux voir que j'ai 120 races de chien pour 10 222 photos. 

In [ ]:
labels_csv["breed"].value_counts().plot.bar(figsize=(20, 5));

Dans le schéma ci-dessus, on peut observer qu'il y a plus de 60 images pour chaque race de chien. C'est une bonne quantité, car pour certains de leurs produits de vision, Google recommande un minimum de 10 images par classe pour commencer. 

Il est temps de récupérer les images et leurs étiquettes

Puisque nous avons les ID des images et leurs étiquettes dans un DataFrame (labels_csv), nous allons l'utiliser pour créer :

* 1) Une liste de chemins d'accès aux images d'entraînement
* 2) Un tableau de toutes les étiquettes
* 3) Un tableau de toutes les étiquettes uniques

Nous allons seulement créer une liste de chemins d'accès aux images plutôt que de les importer toutes pour commencer. En effet, il est beaucoup plus efficace de travailler avec des chemins de fichiers (chaînes de caractères) qu'avec des images.

Comme nous l'utiliserons à plusieurs reprises, nous enregistrerons le chemin d'accès à nos fichiers d'entraînement dans une variable train_path.

In [ ]:
# Define our training file path 
train_path = "../input/dog-breed-identification/train/"

In [ ]:
# Create pathnames from image ID's
filenames = [train_path + fname + ".jpg" for fname in labels_csv["id"]]

# Check the first 10 filenames
filenames[:10]

Maintenant que nous avons une liste de tous les noms de fichiers de la colonne ID de labels_csv, nous pouvons la comparer au nombre de fichiers dans notre répertoire de données d'entraînement pour voir s'ils correspondent.

In [ ]:
if len(os.listdir('/kaggle/input/dog-breed-identification/train/')) == len(filenames):
    print('Number of file matches number of actual images!')
else:
    print('Number of file doesnot matches number of actual images!!')

In [ ]:
#Je retrouve une image par son id
labels_csv['breed'][900]

In [ ]:
# Je regarde l'image directement
from IPython.display import display, Image
Image(filenames[900])

Maintenant que nous avons rassemblé les chemins de fichiers des images, récupérons les étiquettes.
Nous allons les prendre dans labels_csv et les transformer en tableau NumPy.

In [ ]:
import numpy as np
labels = labels_csv["breed"].to_numpy() # convert labels column to NumPy array
labels[:10]

Comparons le nombre d'étiquettes au nombre de noms de fichiers.

In [ ]:
if len(labels) == len(filenames):
    print('Number of labels matches the number of filenames.')
else:
    print('Number of labels doesnot matches the number of filenames')

Si tout a fonctionné, nous devrions avoir la même quantité d'images et d'étiquettes.

Enfin, comme un modèle d'apprentissage automatique ne peut pas prendre de chaînes de caractères en entrée (ce que sont les étiquettes actuellement), nous devons **convertir nos étiquettes en chiffres.**

Pour commencer, nous allons trouver tous les **noms uniques de races de chiens.**

Ensuite, nous allons **parcourir la liste des étiquettes, les comparer aux races uniques** et **créer une liste de booléens** indiquant laquelle est l'étiquette réelle (Vrai) et lesquelles ne le sont pas (Faux).

In [ ]:
unique_breed = np.unique(labels) 
len(unique_breed)

La longueur de unique_breeds doit être de 120, ce qui signifie que nous travaillons avec des images de 120 races de chiens différentes.

Utilisez maintenant unique_breeds pour transformer notre tableau de labels en un tableau de booléens.

In [ ]:
#exemple d'une étiquette transformée en tableau de boléens
print(labels[0])
labels[0] == unique_breed

In [ ]:
# Turn every label into a boolean array
boolean_labels = [label == np.array(unique_breed) for label in labels]
boolean_labels[:2]

un concept important de l'apprentissage automatique consiste à convertir vos données en chiffres avant de les transmettre à un modèle d'apprentissage automatique.

Dans ce cas, nous avons transformé un seul nom de race de chien, tel que boston_bull, en un tableau de chiffres sur le principe du One Hot Encoder.

In [ ]:
# Turining boolean arrays into integers.
print(labels[0])   #orginal index
print(np.where(unique_breed==labels[0]))    #index where labels occurs.
print(boolean_labels[0].argmax())     #index where label occurs in boolean array
print(boolean_labels[0].astype(int))   #there will be a 1 where sample label occurs

Maintenant que nous avons nos étiquettes au format numérique et que les chemins de fichier de nos images sont facilement accessibles (ils ne sont pas encore numériques), nous allons diviser nos données.

# Création de notre propre ensemble de validation

Puisque l'ensemble de données de Kaggle n'est pas fourni avec un ensemble de validation (une division des données sur laquelle nous pouvons tester notre modèle avant de faire des prédictions finales sur l'ensemble de test), nous allons en créer un.

Nous pouvons utiliser la fonction train_test_split de Scikit-Learn ou nous pouvons simplement faire des divisions manuelles des données.

Pour des raisons d'accessibilité ultérieure, sauvegardons nos noms de fichiers en X (données) et nos étiquettes en y.

In [ ]:
# Setup X & y variables
X = filenames
y = boolean_labels

print(f"Number of training images: {len(X)}")
print(f"Number of labels: {len(y)}")

Étant donné que nous travaillons avec plus de 10 000 images, il est préférable de travailler avec une partie d'entre elles pour s'assurer que tout fonctionne bien avant de s'entraîner sur toutes les images.

En effet, le calcul de plus de 10 000 images peut prendre beaucoup de temps. Et notre objectif, lorsque nous travaillons sur des projets d'apprentissage automatique, est de réduire le temps entre les expériences.

Commençons à expérimenter avec 1000 et augmentons ce nombre selon nos besoins.

In [ ]:
#set number of images to set for the experiment.
num_img = 1000 #@param {type:"slider",min:1000,max:10000}

Divisons maintenant nos données en ensembles de formation et de validation. Nous utiliserons une répartition 80/20 (80% de données de formation, 20% de données de validation).

In [ ]:
#let's split our data into train and validation.
from sklearn.model_selection import train_test_split

#spliting into training and validation of total size NUM_IMAGES.

X_train,X_val,y_train,y_val = train_test_split(X[:num_img],
                                                y[:num_img],
                                                test_size=0.2,
                                                random_state=42)
len(X_train),len(X_val),len(y_train),len(y_val)

In [ ]:
# Check out the training data (image file paths and labels)
X_train[:1], y_train[:1]

# Prétraitement des images (transformation des images en tenseurs)


Nos étiquettes sont au format numérique mais nos images ne sont encore que des chemins de fichiers.

Comme nous utilisons TensorFlow, **nos données doivent être sous la forme de tenseurs.**

Un tenseur est un moyen de **représenter des informations sous forme de nombres.** .Un tenseur peut être considéré comme une combinaison de tableaux NumPy, mais avec la capacité spéciale d'être utilisé sur un GPU.

En raison de la façon dont TensorFlow stocke les informations (dans les tenseurs), il permet aux modèles d'apprentissage automatique et d'apprentissage profond d'être **exécutés sur des GPU (généralement plus rapides pour le calcul numérique).**

Pour **prétraiter nos images en Tensors**, nous allons écrire une fonction qui fait plusieurs choses :

* Prend un nom de fichier d'image en entrée.
* Utilise TensorFlow pour lire le fichier et l'enregistrer dans une variable, image.
* Transformer notre image (un fichier jpeg) en tenseurs.
* Redimensionne l'image pour qu'elle soit de forme (224, 224).
* Retourner l'image modifiée.

Un bon endroit pour lire sur ce type de fonction est la documentation TensorFlow sur le **chargement des images.**

Vous vous demandez peut-être pourquoi **(224, 224)**, qui correspond à **(hauteur, largeur)**. C'est parce que c'est la taille de l'entrée que notre modèle (nous le verrons bientôt) prend, une image qui est (224, 224, 3).

**Le 3, c'est le nombre de canaux de couleur par pixel, rouge, vert et bleu (RVB).**

Rendons cela un peu plus concret.

In [ ]:
# Convert image to NumPy array
from matplotlib.pyplot import imread
image = imread(filenames[42]) # read in an image
image.shape

Remarquez la forme de l'image. C'est (257, 350, 3). C'est la hauteur, la largeur, la valeur du canal de couleur.

Et vous pouvez facilement le convertir en un tenseur en utilisant tf.constant().

In [ ]:
tf.constant(image)[:2]

In [ ]:
# Define image size
IMG_SIZE = 224

def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

# Création de lots de données
Maintenant que nous avons une fonction pour convertir nos images en tenseurs, nous allons en construire une fonction pour **transformer nos données en lots** (un TensorFlow BatchDataset).

Qu'est-ce qu'un batch ?

Un lot (également appelé mini-batch) est une petite portion de vos données, disons 32 (32 est généralement la taille de lot par défaut) images et leurs étiquettes. En apprentissage profond, au lieu de trouver des modèles dans un ensemble de données entier en même temps, vous les trouvez souvent un lot à la fois.

Disons que vous avez affaire à plus de 10 000 images (ce qui est le cas). Ensemble, ces fichiers peuvent occuper plus de mémoire que votre GPU. Essayer de les calculer tous entraînerait une erreur.

Il est donc plus efficace de  **créer des lots plus petits de vos données et de les calculer un par un.**

TensorFlow est très efficace lorsque vos données sont des lots de tenseurs (image, étiquette). Nous allons donc construire une fonction pour les créer en premier. Nous allons profiter de la fonction process_image en même temps.

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

Maintenant que nous disposons d'une fonction simple pour transformer les noms des chemins d'accès aux fichiers d'images et les étiquettes associées en tuples (nous les transformerons ensuite en tenseurs), nous allons créer une fonction pour créer des lots de données.

Puisque nous aurons affaire à trois ensembles de données différents (formation, validation et test), nous nous assurerons que la fonction peut s'adapter à chaque ensemble.

Nous définirons une taille de lot par défaut de 32.

In [ ]:
# Define the batch size, 32 is a good default
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch
  
  # If the data if a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    # If the data is a training dataset, we shuffle it
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                              tf.constant(y))) # labels
    
    # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(x))

    # Create (image, label) tuples (this also turns the image path into a preprocessed image)
    data = data.map(get_image_label)

    # Turn the data into batches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
# Check out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

Nous avons nos données en lots, plus précisément, elles sont dans des paires Tensor de (images, étiquettes) prêtes à être utilisées sur un GPU.

# Visualisation des lots de données

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in a data batch
def show_25_images(images, labels):
  """
  Displays 25 images from a data batch.
  """
  # Setup the figure
  plt.figure(figsize=(10, 10))
  # Loop through 25 (for displaying 25 images)
  for i in range(25):
    # Create subplots (5 rows, 5 columns)
    ax = plt.subplot(5, 5, i+1)
    # Display an image
    plt.imshow(images[i])
    # Add the image label as the title
    plt.title(unique_breed[labels[i].argmax()])
    # Turn gird lines off
    plt.axis("off")

Pour rendre le calcul efficace, un lot est une collection de tenseurs fortement enroulée.
Ainsi, pour visualiser les données d'un lot, nous devons le dérouler. Nous pouvons le faire en appelant la méthode as_numpy_iterator() sur un lot de données. Cela transformera notre lot de données en quelque chose qui peut être itéré. Le passage d'un itérable à next() renverra l'élément suivant dans l'itérateur. Dans notre cas, next renverra un lot de 32 images et paires d'étiquettes.

In [ ]:
# Visualize training images from the training data batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
# Visualize validation images from the validation data batch
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

# Création et formation d'un modèle

Maintenant que nos données sont prêtes, préparons leur modélisation. 

# Comment choisir un modèle ?

Empilements de couches Conv2D et MaxPool2D:
MaxPool2D réduit la taille des cartes de caractéristiques
-  Le maximum pooling est appliqué après l'activation ReLU et a pour effet d'"intensifier" les caractéristiques. L'étape de mise en commun augmente la proportion de pixels actifs par rapport aux pixels nuls.
- filters= On  indique à la couche convolutionnelle le nombre de cartes de caractéristiques que vous souhaitez qu'elle crée en sortie.
- padding='same'. remplissage pour obtenir des map feature de même taille que l’image.
- kernel_size = Dimensions du noyau.
- L'activation ReLU dit que les valeurs négatives ne sont pas importantes et les met donc à 0. 
- Dropout Corrige l’overfitting. Réduit le bruit en éliminant aléatoirement une fraction des unités d’entrées de couches. 
- Early stopping - Arrêt précoce si la perte de validation (validation loss) ne descend plus. Bcp de bruit peu de signal. 
- Softmax = problème de classification multiclasse


# Construction d'un modèle

Avant de construire un modèle, nous devons définir quelques éléments :

La forme d'entrée (images, sous la forme de tenseurs) de notre modèle.
La forme de sortie (étiquettes d'images, sous forme de tenseurs) de notre modèle.
L'URL du modèle que nous voulons utiliser.
Ces éléments constituent une pratique standard, quel que soit le modèle d'apprentissage automatique que vous utilisez.

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_breed) # number of unique labels

# Data augmentation

ImageDataGenerator est la classe de référence de Keras pour le pipelining des données d'image pour l'apprentissage en profondeur. Il permet un accès facile à votre système de fichiers local et à plusieurs méthodes différentes pour charger des données à partir de différentes structures. Il possède également des capacités de prétraitement et d'augmentation des données assez puissantes.

https://ichi.pro/fr/methodes-de-flux-de-keras-imagedatagenerator-et-quand-les-utiliser-203621254419857

In [ ]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3) # stops after 3 rounds of no improvements
NUM_EPOCHS = 100

In [ ]:
import keras
import tensorflow as tf
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import Adam


model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape = INPUT_SHAPE[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(OUTPUT_SHAPE, activation='softmax'))

In [ ]:
  # Compile the model
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # A friend telling our model how to improve its guesses
      metrics=["accuracy"] # We'd like this to go up
  )

# Build the model
model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting

Maintenant que nous avons accès à nos données, nous pouvons mettre en place un ImageDataGenerator et connectez -vous à notre source de données à l' aide de notre première méthode, .flow.  

ImageDataGenerator:

- **rescale**(facultatif, mais recommandé) - L'un des nombreux paramètres d'augmentation ajuste les valeurs de pixels de notre image. Le réglage rescale=1./255ajustera nos valeurs de pixels entre 0 et 1.

- **validation_split**(facultatif, mais recommandé) - Attribue un train-test-split aux données passées en fonction du flottant fourni. L'utilisation de cette méthode signifie que vous devez attribuer un sous-ensemble de formation et de validation dans votre .flowméthode. Le réglage validation_split=0.2nous donne une répartition des essais de train de 80/20.

- **x_train, y_train(obligatoire)** - Les deux premiers arguments passés à votre méthode de flux seront toujours vos données X et y sous la forme de tableaux NumPy.

- **batch_size(obligatoire)** - Nombre d'images à inclure dans chaque lot de données d'entraînement. N'oubliez pas que les réseaux de neurones s'entraînent sur une série d' époques , au cours desquelles des groupes d'images appelés lots sont transmis à votre modèle.

- **shuffle**(facultatif, mais recommandé pour le générateur de formation) - indique à votre générateur s'il doit ou non sélectionner au hasard les images à intégrer dans chaque lot. L'avantage de ceci est que notre modèle ne sera pas formé sur plusieurs images de la même catégorie dos à dos. Nous ne l'incluons pas dans notre générateur de validation car nous souhaitons pouvoir interpréter facilement nos prédictions.

- **seed** (facultatif) - Définit un état aléatoire pour la reproductibilité.

- **subset**(obligatoire quand validation_splitest défini) - Indique quel sous-ensemble de données est contenu dans le générateur, correspond directement à l' validation_splitargument passé au constructeur IDG.

In [ ]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [ ]:
from os.path import isfile, join, abspath, exists, isdir, expanduser
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical

#Extracting different classes
dog_breeds = sorted(labels_csv['breed'].unique())
n_classes = len(dog_breeds)

#Converting classes to numbers
class_to_num = dict(zip(dog_breeds,range(n_classes)))

image_size = (224,224,3)
data_dir = "../input/dog-breed-identification/train"
"""Return arrays that represents the images and processed target given where the images are saved, the dataframe that contains the id and the breeds and the image size"""
image_names = labels_csv['id']
image_labels = labels_csv['breed']
data_size = len(image_names)

X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
y = np.zeros([data_size,1],dtype = np.uint8)

for i in range(data_size):
    img_name = image_names[i]
    img_dir = join(data_dir,img_name+'.jpg')
    img_pixels = load_img(img_dir,target_size=image_size)
    X[i] = img_pixels
    y[i] = class_to_num[image_labels[i]]

y = to_categorical(y)

ind = np.random.permutation(data_size)
X = X[ind]
y = y[ind]
print('Ouptut Data Size: ', X.shape)
print('Ouptut Label Size: ', y.shape)
 

In [ ]:
#spliting into training and validation of total size NUM_IMAGES.

X_train,X_val,y_train,y_val = train_test_split(X[:num_img],
                                                y[:num_img],
                                                test_size=0.2,
                                                random_state=42)
len(X_train),len(X_val),len(y_train),len(y_val)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)


train_generator = datagen.flow(X_train,
                               y_train,
                               batch_size=32,
                               shuffle=True,
                               seed=42,
                               subset='training')

valid_generator = datagen.flow(X_val,
                               y_val,
                               batch_size=32,
                               seed=42,
                               subset='validation')


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

history = model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
           epochs=NUM_EPOCHS,
            validation_freq=1,
            callbacks=early_stopping)

In [ ]:
# Check again if GPU is available (otherwise computing will take a looooonnnnggggg time)
print("GPU", "available (OUUUUIIII!!!!)" if tf.config.list_physical_devices("GPU") else "NONNNNNN!!!!!")

In [ ]:
# Make predictions on the validation data (not used to train on)
predictions = model.predict(val_data, verbose=1) # verbose shows us how long there is to go
predictions

In [ ]:
# Check the shape of predictions
predictions.shape

La réalisation de prédictions avec notre modèle renvoie un tableau avec une valeur différente pour chaque étiquette.

Dans ce cas, faire des prédictions sur les données de validation (200 images) renvoie un tableau (prédictions) de tableaux, chacun contenant 120 valeurs différentes (une pour chaque race de chien unique).

Ces différentes valeurs sont les probabilités ou la probabilité que le modèle ait prédit qu'une certaine image était une certaine race de chien. Plus la valeur est élevée, plus le modèle pense qu'une image donnée est une race de chien spécifique.

Voyons comment convertir un tableau de probabilités en une étiquette réelle.

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
history_df.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_breed[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[0])
pred_label

Rappelez-vous, le modèle n'a pas été entraîné sur les données de validation, pendant la fonction fit(), il a seulement utilisé les données de validation pour s'évaluer. Nous pouvons donc utiliser les images de validation pour comparer visuellement les prédictions de notre modèle avec les étiquettes de validation.

Puisque nos données de validation (val_data) sont sous forme de lot, pour obtenir une liste d'images et d'étiquettes de validation, nous devrons les débloquer (en utilisant unbatch()) et ensuite les transformer en itérateur en utilisant as_numpy_iterator().


In [ ]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and returns separate arrays
  of images and labels.
  """
  images = []
  labels = []
  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_breed[np.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

Maintenant, nous avons des moyens d'obtenir :

* Des étiquettes de prédiction
* Étiquettes de validation (étiquettes de vérité)
* Images de validation
* Créons quelques fonctions pour rendre tout cela un peu plus visuel.

Plus précisément, nous voulons être en mesure de visualiser une image, son étiquette prédite et son étiquette réelle (true label).

La première fonction que nous allons créer va :

* Prendre un tableau de probabilités de prédiction, un tableau d'étiquettes réelles, un tableau d'images et un nombre entier.
* Convertir les probabilités de prédiction en une étiquette prédite.
* Tracer l'étiquette prédite, sa probabilité prédite, l'étiquette de vérité et l'image cible sur un seul graphique.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth label and image for sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
  
  # Get the pred label
  pred_label = get_pred_label(pred_prob)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% ({})".format(pred_label,
                                      np.max(pred_prob)*100,
                                      true_label),
                                      color=color)

In [ ]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          images=val_images)

Comme nous travaillons sur un problème multi-classes (120 races de chiens différentes), il serait également intéressant de voir quelles autres prédictions notre modèle fait. Plus précisément, si notre modèle prédit une certaine étiquette avec une probabilité de 24 %, que prédit-il d'autre ?

Construisons une fonction pour le démontrer. Cette fonction va :
**Prendre en entrée un tableau de probabilités de prédiction, un tableau d'étiquettes de vérité terrain et un nombre entier./
Trouver l'étiquette prédite en utilisant get_pred_label()./
Trouver les 10 premiers :
Indices de probabilités de prédiction - 
Valeurs des probabilités de prédiction - 
Étiquettes de prédiction /. 
Tracez les 10 premières valeurs de probabilité de prédiction et les étiquettes, en colorant la véritable étiquette en vert.**

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plots the top 10 highest prediction confidences along with
  the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
  # Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]
  # Find the top 10 prediction labels
  top_10_pred_labels = unique_breed[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)), 
                     top_10_pred_values, 
                     color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,
               n=9)

In [ ]:
# Let's check a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(5*2*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions,
                labels=val_labels,
                n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

https://www.kaggle.com/mrdbourke/tensorflow-2-x-tensorflow-hub-end-to-end-example